In [69]:
import pandas as pd
import plotly.express as px
import numpy as np
from mainnet_launch.constants import *
from mainnet_launch.database.views import get_token_details_dict, 

from mainnet_launch.data_fetching.fetch_data_from_3rd_party_api import (
    make_many_requests_to_3rd_party,
    make_single_request_to_3rd_party,
    THIRD_PARTY_SUCCESS_KEY,
)



def add_actual_prices(df: pd.DataFrame) -> pd.DataFrame:
    token_to_decimals, token_to_symbol = get_token_details_dict()
    chain_ids_to_names = {c.chain_id: c.name for c in ALL_CHAINS}

    # normalize amounts safely (skip NaN)
    def norm(amount, token):
        if pd.isna(amount) or token not in token_to_decimals:
            return np.nan
        try:
            return int(amount) / (10 ** token_to_decimals[token])
        except Exception:
            return np.nan

    df["buy_amount_norm"] = df.apply(lambda r: norm(r["buyAmount"], r["buyToken"]), axis=1)
    df["sell_amount_norm"] = df.apply(lambda r: norm(r["sellAmount"], r["sellToken"]), axis=1)

    df["buy_symbol"] = df["buyToken"].map(token_to_symbol)
    df["sell_symbol"] = df["sellToken"].map(token_to_symbol)

    # ratio of normalized buy/sell
    df["buy_amount_price"] = df["buy_amount_norm"] / df["sell_amount_norm"]
    df["label"] = df["sell_symbol"] + " -> " + df["buy_symbol"]

    # handle missing price columns gracefully
    for col in ["buy_token_price", "sell_token_price"]:
        if col not in df.columns:
            df[col] = np.nan

    df["safe_value_bought"] = df["buy_token_price"] * df["buy_amount_norm"]
    df["safe_value_sold"] = df["sell_token_price"] * df["sell_amount_norm"]

    df["safe_value_slippage_bps"] = 1000 * (df["safe_value_sold"] - df["safe_value_bought"]) / df["safe_value_sold"]
    df["chain_name"] = df["chainId"].map(chain_ids_to_names)
    df["long_label"] = (
        df["label"] + " " + df["sell_amount_norm"].fillna(0).astype(int).astype(str) + " " + df["chain_name"]
    )
    return df


save_path = "/Users/pb/Documents/Github/Tokemak/v2-rebalance-dashboard/working_data/swap_matrix_prices2/all_autopools_full_swap_matrix_with_prices2.csv"

df = add_actual_prices(pd.read_csv(save_path, low_memory=False))
df["long_label"]

SyntaxError: trailing comma not allowed without surrounding parentheses (3149079236.py, line 5)

In [61]:
df.groupby("chain_name")["3rd_party_response_success"].value_counts().reset_index()

,chain_name,3rd_party_response_success,count
0,arb,False,12
1,base,False,42
2,eth,True,482
3,eth,False,260
4,plasma,False,12
5,sonic,True,20


In [62]:
valid_response = {
    "3rd_party_response_success": True,
    "aggregatorName": "Odos",
    "asyncSwapper": "0xC15bd07f650e18DCF2E41f10F525f86232B2cBAe",
    "buyAmount": "2084630835071501205504",
    "datetime_received": pd.Timestamp("2025-10-09 21:26:23.980214+0000", tz="UTC"),
    "expiration": 9999999999,
    "fullQuoteDetails": {
        "allQuotes": [
            {"buyAmount": None, "minBuyAmount": None, "name": "0xV2", "succeeded": False},
            {"buyAmount": None, "minBuyAmount": None, "name": "daiUsds", "succeeded": False},
            {"buyAmount": None, "minBuyAmount": None, "name": "4626", "succeeded": False},
            {
                "buyAmount": "2084630835071501205504",
                "minBuyAmount": "1042315417535750602752",
                "name": "Odos",
                "succeeded": True,
            },
        ],
        "quote": {
            "blockNumber": 23542853,
            "dataGasEstimate": 0,
            "gasEstimate": 4887776,
            "gasEstimateValue": 18.41701231708594,
            "gweiPerGas": 0.87,
            "inAmounts": ["10000000000000000000000"],
            "inTokens": ["0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"],
            "inValues": [43309660.21572473],
            "netOutValue": 8839120.962392136,
            "outAmounts": ["2084630835071501205504"],
            "outTokens": ["0x04c154b66cb340f3ae24111cc767e0184ed00cc6"],
            "outValues": [8839139.379404454],
            "partnerFeePercent": 0,
            "pathId": "b9697976b08f67eeb7b28b86569cd9a8",
            "pathViz": None,
            "percentDiff": -79.58471187966214,
            "permit2Hash": None,
            "permit2Message": None,
            "priceImpact": 79.5662599488833,
            "traceId": "90234f53-69f0-4df9-b331-ff32ae2a6936",
        },
        "tx": {
            "blockNumber": 23542853,
            "deprecated": None,
            "gasEstimate": 4887776,
            "gasEstimateValue": 18.41701231708594,
            "inputTokens": [
                {"amount": "10000000000000000000000", "tokenAddress": "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"}
            ],
            "netOutValue": 8839120.962392136,
            "outValues": ["8839139.379404454"],
            "outputTokens": [
                {"amount": "2084630835071501205504", "tokenAddress": "0x04c154b66cb340f3ae24111cc767e0184ed00cc6"}
            ],
            "simulation": None,
            "traceId": "204c0b30-452f-4e67-9758-573bb56e98d2",
            "transaction": {
                "chainId": 1,
                "data": "0x3b635ce4000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc20000000000000000000000000000000000000000000000000000000000000000000000000000000000000000365084b05fa7d5028346bd21d842ed0601bab5b800000000000000000000000004c154b66cb340f3ae24111cc767e0184ed00cc600000000000000000000000000000000000000000000007102107ba666c0000000000000000000000000000000000000000000000000003881083dd333620000000000000000000000000000000000000000000000000000000000000000dead0000000000000000000000000000000000000000000000000000000000000140000000000000000000000000365084b05fa7d5028346bd21d842ed0601bab5b8000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003900b071d011b15a09906010001001327534901000102010162634e030200010302001e0100957d3d0d0200040200010204757b070300020532296969ef14eb0c6d29669c550d4a04491302300002000000000000000000800125adcb4a070300020593d199263632a4ef4bb438f1feb99e57b4b5f0bd0000000000000000000005c2012bd114d2680300060205070801353b89bc6803000902050a0b010c6b52630d03000c0200011762bd564a04000d020001010331ce50670500010e020100014ada8e28070600020f88794c65550deb6b4087b7552ecf29511379441000000000000000000000064801f68dd69a67060001100200010104c6e7e9510600110200080dd7d7eb16458b00000d06001202000400000a0002064c0000030d8ed9b24d000348aef4fd4a00001314000102560015141600010941adee714a020017180100084a0000191801000056061a160f010004670600011b1c00010c020fff00a2881a90bf33f03e7a3f803765cd2ed5c8928dfbc02aaa39b223fe8d0a0e5c4f27ead9083c756cc24028daac072e492d34a3afdbef0ba7e35d8b55c463818bbdd21e69be108a23ac1e84cbf66399bd7d7f39c581f595b53c5cb19bd0b3f8da6c935e2ca06b31a94029fd7840d780191b6d63fa0d269bd88390551c1795392094fe6d29b758eccd233cfaa2602411802d8bea09be0af8fd8d08314a63e706b29cc4ce391d82d164c166df9c8336ddf84206b2f8120fe906e030a44ef24ca8c7dc7b7c53a6c4f00ce9775f661b0bd1739349b9a2a3ef60be277c5d2d29109830a1aaad605bbf02a9dfa7b0b92ec2fb7daa9c3b46c0ceb5b9e304fcd6d88fc50f7dd24b31bc8efd02a0a40545f32dba5d664cbbc1570d3fedf604c154b66cb340f3ae24111cc767e0184ed00cc6c8eb2cf2f792f77af0cd9e203305a585e588179d8dd893b9ec19d89b66c01794cf5e04e30d284e5302edd21f1db8eb146be60998e9801691e725f11921e27a5e5513d6e65c4f830167390997aa84843a0000000000000000000000000000000000000000dc24316b9ae028f1497c275eb9192a3ea0f67022ae7ab96520de3a18e5e111b5eaab095312d7fe84b657b895b265c38c53fff00166cf7f6a3c70587d5e8422345238f34275888049021821e8e08caa1f4d9f9d15101eec665f77210cb999639f760f831e6951bdc4734b9f7f3e1b74afebc670c736a0edb630bf3e17cad0baf1d6b64079ec219808d2708feb0100546f2cd4c9d97f798ffc9755e47865ff7ee600000000000000000000000000000000",
                "from": "0x000000000000000000000000000000000000dEaD",
                "gas": 9775552,
                "gasPrice": 870000000,
                "nonce": 0,
                "to": "0xCf5540fFFCdC3d510B18bFcA6d2b9987b0772559",
                "value": "0",
            },
        },
    },
    "internal": False,
    "minBuyAmount": "1042315417535750602752",
    "request_kwargs": {
        "json": {
            "buyToken": "0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6",
            "chainId": 1,
            "excludeSources": "Bebop,Lifi",
            "includeSources": "",
            "returnAll": True,
            "sellAll": True,
            "sellAmount": "10000000000000000000000",
            "sellToken": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2",
            "slippageBps": 5000,
            "systemName": "gen3",
            "taker": "0x000000000000000000000000000000000000dEaD",
            "timeoutMS": 20000,
        },
        "method": "POST",
        "url": "https://swaps-pricing.tokemaklabs.com/swap-quote-v2",
    },
    "tx": {
        "data": "0x3b635ce4000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc20000000000000000000000000000000000000000000000000000000000000000000000000000000000000000365084b05fa7d5028346bd21d842ed0601bab5b800000000000000000000000004c154b66cb340f3ae24111cc767e0184ed00cc600000000000000000000000000000000000000000000007102107ba666c0000000000000000000000000000000000000000000000000003881083dd333620000000000000000000000000000000000000000000000000000000000000000dead0000000000000000000000000000000000000000000000000000000000000140000000000000000000000000365084b05fa7d5028346bd21d842ed0601bab5b8000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003900b071d011b15a09906010001001327534901000102010162634e030200010302001e0100957d3d0d0200040200010204757b070300020532296969ef14eb0c6d29669c550d4a04491302300002000000000000000000800125adcb4a070300020593d199263632a4ef4bb438f1feb99e57b4b5f0bd0000000000000000000005c2012bd114d2680300060205070801353b89bc6803000902050a0b010c6b52630d03000c0200011762bd564a04000d020001010331ce50670500010e020100014ada8e28070600020f88794c65550deb6b4087b7552ecf29511379441000000000000000000000064801f68dd69a67060001100200010104c6e7e9510600110200080dd7d7eb16458b00000d06001202000400000a0002064c0000030d8ed9b24d000348aef4fd4a00001314000102560015141600010941adee714a020017180100084a0000191801000056061a160f010004670600011b1c00010c020fff00a2881a90bf33f03e7a3f803765cd2ed5c8928dfbc02aaa39b223fe8d0a0e5c4f27ead9083c756cc24028daac072e492d34a3afdbef0ba7e35d8b55c463818bbdd21e69be108a23ac1e84cbf66399bd7d7f39c581f595b53c5cb19bd0b3f8da6c935e2ca06b31a94029fd7840d780191b6d63fa0d269bd88390551c1795392094fe6d29b758eccd233cfaa2602411802d8bea09be0af8fd8d08314a63e706b29cc4ce391d82d164c166df9c8336ddf84206b2f8120fe906e030a44ef24ca8c7dc7b7c53a6c4f00ce9775f661b0bd1739349b9a2a3ef60be277c5d2d29109830a1aaad605bbf02a9dfa7b0b92ec2fb7daa9c3b46c0ceb5b9e304fcd6d88fc50f7dd24b31bc8efd02a0a40545f32dba5d664cbbc1570d3fedf604c154b66cb340f3ae24111cc767e0184ed00cc6c8eb2cf2f792f77af0cd9e203305a585e588179d8dd893b9ec19d89b66c01794cf5e04e30d284e5302edd21f1db8eb146be60998e9801691e725f11921e27a5e5513d6e65c4f830167390997aa84843a0000000000000000000000000000000000000000dc24316b9ae028f1497c275eb9192a3ea0f67022ae7ab96520de3a18e5e111b5eaab095312d7fe84b657b895b265c38c53fff00166cf7f6a3c70587d5e8422345238f34275888049021821e8e08caa1f4d9f9d15101eec665f77210cb999639f760f831e6951bdc4734b9f7f3e1b74afebc670c736a0edb630bf3e17cad0baf1d6b64079ec219808d2708feb0100546f2cd4c9d97f798ffc9755e47865ff7ee600000000000000000000000000000000",
        "from": "0x000000000000000000000000000000000000dEaD",
        "gasPrice": "10264329",
        "to": "0xCf5540fFFCdC3d510B18bFcA6d2b9987b0772559",
        "value": "0",
    },
}

In [ ]:
def _flatten_all_quotes(tokemak_response: dict) -> dict:
    """Get the buy amount min buy amount and succeeded for each quote source"""
    all_quotes = tokemak_response.get("fullQuoteDetails", {}).get("allQuotes", [])
    other_quotes = {}
    for minimal_quote in all_quotes:
        other_quotes[f'{minimal_quote["name"]}_buyAmount'] = minimal_quote["buyAmount"]
        other_quotes[f'{minimal_quote["name"]}_succeeded'] = minimal_quote["succeeded"]
        other_quotes[f'{minimal_quote["name"]}_minBuyAmount'] = minimal_quote["minBuyAmount"]

    return other_quotes


def _extract_request_kwargs(tokemak_response: dict) -> dict:
    """Extract the request kwargs from the dict"""
    request_kwargs = tokemak_response["request_kwargs"]
    json_payload = request_kwargs.pop("json")
    request_kwargs.update(json_payload)
    return request_kwargs


def _extract_top_level_keys(response: dict) -> dict:
    top_level_keys = [
        "3rd_party_response_success",
        "aggregatorName",
        "asyncSwapper",
        "buyAmount",
        "datetime_received",
        "expiration",
        "internal",
        "minBuyAmount",
    ]

    return {k: response.get(k) for k in top_level_keys}


def _extract_block_number(tokemak_response: dict) -> dict:
    return {"block": tokemak_response["fullQuoteDetails"]["tx"]["blockNumber"]}


def _handle_successful_tokemak_request(tokemak_response: dict) -> dict:
    processed = {}
    processed.update(_extract_block_number(tokemak_response))
    processed.update(_extract_top_level_keys(tokemak_response))
    processed.update(_flatten_all_quotes(tokemak_response))
    processed.update(_extract_request_kwargs(tokemak_response))
    return processed


def _extract_invalid_response_top_level_keys(response: dict) -> dict:
    expected_keys = ["3rd_party_response_success", "body", "datetime_received", "headers", "status", "url"]

    return {k: response.get(k) for k in expected_keys}


def _handle_failed_tokemak_request(tokemak_response: dict) -> dict:
    """Process a failed tokemak response into a flat dict"""
    processed = {}
    processed.update(_extract_invalid_response_top_level_keys(tokemak_response))
    processed.update(_extract_request_kwargs(tokemak_response))
    return processed


def _process_tokemak_response(tokemak_response: dict) -> dict:
    """Process the tokemak response into a flat dict"""
    if tokemak_response[THIRD_PARTY_SUCCESS_KEY]:
        return _handle_successful_tokemak_request(tokemak_response)
    else:
        return _handle_failed_tokemak_request(tokemak_response)

{'block': 23542853,
 '3rd_party_response_success': True,
 'aggregatorName': 'Odos',
 'asyncSwapper': '0xC15bd07f650e18DCF2E41f10F525f86232B2cBAe',
 'buyAmount': '2084630835071501205504',
 'datetime_received': Timestamp('2025-10-09 21:26:23.980214+0000', tz='UTC'),
 'expiration': 9999999999,
 'internal': False,
 'minBuyAmount': '1042315417535750602752',
 '0xV2_buyAmount': None,
 '0xV2_succeeded': False,
 '0xV2_minBuyAmount': None,
 'daiUsds_buyAmount': None,
 'daiUsds_succeeded': False,
 'daiUsds_minBuyAmount': None,
 '4626_buyAmount': None,
 '4626_succeeded': False,
 '4626_minBuyAmount': None,
 'Odos_buyAmount': '2084630835071501205504',
 'Odos_succeeded': True,
 'Odos_minBuyAmount': '1042315417535750602752',
 'method': 'POST',
 'url': 'https://swaps-pricing.tokemaklabs.com/swap-quote-v2',
 'buyToken': '0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6',
 'chainId': 1,
 'excludeSources': 'Bebop,Lifi',
 'includeSources': '',
 'returnAll': True,
 'sellAll': True,
 'sellAmount': '100000000000000